# Actor-Critic

Теорема о градиенте стратегии связывает градиент целевой функции  и градиент самой стратегии:

$$\nabla_\theta J(\theta) = \mathbb{E}_\pi [Q^\pi(s, a) \nabla_\theta \ln \pi_\theta(a \vert s)]$$

Встает вопрос, как оценить $Q^\pi(s, a)$? В чистом policy-based алгоритме REINFORCE используется отдача $G_t$, полученная методом Монте-Карло в качестве несмещенной оценки $Q^\pi(s, a)$. В Actor-Critic же предлагается отдельно обучать нейронную сеть Q-функции — критика.

Актор-критиком часто называют обобщенный фреймворк (подход), нежели какой-то конкретный алгоритм. Как подход актор-критик не указывает, каким конкретно [policy gradient] методом обучается актор и каким [value based] методом обучается критик. Таким образом актор-критик задает целое [семейство](https://proceedings.neurips.cc/paper_files/paper/1999/file/6449f44a102fde848669bdd9eb6b76fa-Paper.pdf) различных алгоритмов. Рекомендую в качестве шпаргалки использовать упомянутый в тетрадке с REINFORCE [пост из блога Lilian Weng](https://lilianweng.github.io/posts/2018-04-08-policy-gradient/), посвященный наиболее популярным алгоритмам семейства актор-критиков

В данной тетрадке познакомимся с наиболее простым вариантом актор-критика, который так и называют Actor-Critic:

In [17]:
# Cтавим нужные зависимости, если это колаб
try:
    import google.colab
    COLAB = True
except ModuleNotFoundError:
    COLAB = False
    pass

if COLAB:
    !pip -q install "gymnasium[classic-control, atari, accept-rom-license]"
    !pip -q install piglet
    !pip -q install imageio_ffmpeg
    !pip -q install moviepy==1.0.3

In [18]:
from collections import deque

import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.distributions import Categorical

%matplotlib inline

In [19]:
env = gym.make("CartPole-v1")
env.reset()

print(f'{env.observation_space=}')
print(f'{env.action_space=}')

n_actions = env.action_space.n
state_dim = env.observation_space.shape
print(f'Action_space: {n_actions} | State_space: {env.observation_space.shape}')

env.observation_space=Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
env.action_space=Discrete(2)
Action_space: 2 | State_space: (4,)


(1 балл)

In [24]:
def select_action_eps_greedy(Q, state, epsilon):
    """Выбирает действие epsilon-жадно."""
    if not isinstance(state, torch.Tensor):
        state = torch.tensor(state, dtype=torch.float32)
    Q_s = Q(state).detach().numpy()

    if np.random.uniform(0, 1) < epsilon:
        action = np.random.randint(len(Q_s))
    else:
        action = np.argmax(Q_s)

    action = int(action)
    return action

def sample_batch(replay_buffer, n_samples):
    indices = np.random.choice(len(replay_buffer), min(n_samples, len(replay_buffer)), replace=False)
    batch = [replay_buffer[i] for i in indices]

    states = [exp[0] for exp in batch]
    actions = [exp[1] for exp in batch]
    rewards = [exp[2] for exp in batch]
    next_states = [exp[3] for exp in batch]
    terminateds = [exp[4] for exp in batch]

    return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(terminateds)

## Shared-body Actor-Critic

Актор и критик могут обучаться в разных режимах — актор только on-policy (шаг обучения на текущей собранной подтраектории), а критик on-policy или off-policy (шаг обучения на текущей подтраектории или на батче из replay buffer). Это с одной стороны привносит гибкость в обучение, с другой — усложняет его.

Если актор и критик оба обучаются on-policy, то имеет смысл объединить их сетки в одну и делать общий шаг обратного распространения ошибки. Однако, если они обучаются в разных режимах (и с разной частотой обновления), то велика вероятность, что их шаги обучения могут начать конфликтовать в случае общего тела — для такого варианта намного предпочтительнее разделить их на разные подсети (либо аккуратно настраивать гиперпарметры, чтобы стабилизировать обучение). В целом, рекомендуется использовать общий энкодер наблюдений, а далее как можно скорее разделять головы.

Сделаем реализацию актор-критика с общим телом и с on-policy вариантом обучения.

In [25]:
class ActorBatch:
    def __init__(self):
        self.logprobs = []
        self.q_values = []

    def append(self, log_prob, q_value):
        self.logprobs.append(log_prob)
        self.q_values.append(q_value)

    def clear(self):
        self.logprobs.clear()
        self.q_values.clear()

(3 балла)

In [39]:
class ActorCriticModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim):
        super().__init__()

        layers = []
        prev_dim = input_dim
        for dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, dim))
            layers.append(nn.ReLU())
            prev_dim = dim

        self.net = nn.Sequential(*layers)
        self.actor_head = nn.Sequential(
            nn.Linear(prev_dim, output_dim),
            nn.Softmax(dim=-1)
        )
        self.critic_head = nn.Linear(prev_dim, output_dim)

    def forward(self, state):
        features = self.net(state)
        action_probs = self.actor_head(features)
        dist = Categorical(action_probs)

        action = dist.sample()
        log_prob = dist.log_prob(action)
        Q_s_a = self.critic_head(features).gather(1, action.unsqueeze(1)).squeeze(1)

        return action, log_prob, Q_s_a

    def evaluate(self, state):
        q_values = self.critic_head(self.net(state))
        return q_values

(6 баллов)

In [42]:
from torch import optim
class ActorCriticAgent:
    def __init__(self, state_dim, action_dim, hidden_dims, lr, gamma, critic_rb_size):
        self.lr = lr
        self.gamma = gamma

        self.actor_critic = ActorCriticModel(state_dim, hidden_dims, action_dim)
        self.opt = optim.Adam(self.actor_critic.parameters(), lr=lr)

        self.actor_batch = ActorBatch()
        self.critic_rb = deque(maxlen=critic_rb_size)

    def act(self, state):
        state_tensor = to_tensor(state).view(1, -1)
        action, log_prob, q_value = self.actor_critic(state_tensor)
        self.actor_batch.append(log_prob, q_value.detach())
        return action.item()

    def append_to_replay_buffer(self, s, a, r, next_s, terminated):
        self.critic_rb.append((s, a, r, next_s, terminated))

    def evaluate(self, state):
        return self.actor_critic.evaluate(state)

    def update(self, rollout_size, critic_batch_size, critic_updates_per_actor):
        if len(self.actor_batch.q_values) < rollout_size:
            return

        self.opt.zero_grad()
        critic_loss = self.update_critic(critic_batch_size, critic_updates_per_actor)
        actor_loss = self.update_actor()
        loss = critic_loss + actor_loss
        loss.backward()

        self.opt.step()
        self.actor_batch.clear()

    def update_actor(self):
        Q_s_a = to_tensor(self.actor_batch.q_values)
        logprobs = torch.stack(self.actor_batch.logprobs)
        advantages = (Q_s_a - Q_s_a.mean()).detach()
        loss = -(logprobs.squeeze() * advantages.squeeze()).mean()
        return loss

    def update_critic(self, batch_size, n_updates=1):
        total_loss = torch.tensor(0.0, requires_grad=True)
        update_count = 0

        for _ in range(n_updates):
            if len(self.critic_rb) < batch_size:
                continue

            states, actions, rewards, next_states, terminateds = sample_batch(self.critic_rb, batch_size)
            loss = self.compute_td_loss(states, actions, rewards, next_states, terminateds)
            total_loss = loss + total_loss
            update_count += 1

        return total_loss / update_count

    def compute_td_loss(
        self, states, actions, rewards, next_states, terminateds, regularizer=0.1
    ):
        # переводим входные данные в тензоры
        s = to_tensor(states)                     # shape: [batch_size, state_size]
        a = to_tensor(actions, int).long()        # shape: [batch_size]
        r = to_tensor(rewards)                    # shape: [batch_size]
        s_next = to_tensor(next_states)           # shape: [batch_size, state_size]
        term = to_tensor(terminateds, bool)       # shape: [batch_size]

        # получаем Q[s, a] для выбранных действий в текущих состояниях (для каждого примера из батча)
        Q_s_a = self.actor_critic.evaluate(s).gather(1, a.unsqueeze(1)).squeeze(1)

        with torch.no_grad():
            Q_sn = self.actor_critic.evaluate(s_next)
            V_sn = Q_sn.max(dim=1)[0]

        # вычисляем TD target и далее TD error
        target = r + self.gamma * V_sn * (~term)
        td_error = target - Q_s_a

        # MSE loss для минимизации
        loss = torch.mean(td_error ** 2)
        # добавляем регуляризацию на значения Q
        loss += regularizer * Q_s_a.mean()
        return loss

In [43]:
def run_actor_critic(
        env_name="CartPole-v1",
        hidden_dims=(128, 128), lr=5e-4,
        total_max_steps=200_000,
        train_schedule=16, replay_buffer_size=50000, batch_size=64, critic_updates_per_actor=4,
        eval_schedule=1000, smooth_ret_window=10, success_ret=200.
):
    env = gym.make(env_name)
    episode_return_history = deque(maxlen=smooth_ret_window)

    agent = ActorCriticAgent(
        state_dim=env.observation_space.shape[0], action_dim=env.action_space.n, hidden_dims=hidden_dims,
        lr=lr, gamma=.995, critic_rb_size=replay_buffer_size
    )

    s, _ = env.reset()
    done, episode_return = False, 0.
    eval = False

    for global_step in range(1, total_max_steps+1):
        a = agent.act(s)
        s_next, r, terminated, truncated, _ = env.step(a)
        episode_return += r
        done = terminated or truncated

        # train step
        agent.append_to_replay_buffer(s, a, r, s_next, terminated)
        agent.update(train_schedule, batch_size, critic_updates_per_actor)

        # evaluate
        if global_step % eval_schedule == 0:
            eval = True

        s = s_next
        if done:
            if eval:
                episode_return_history.append(episode_return)
                avg_return = np.mean(episode_return_history)
                print(f'{global_step=} | {avg_return=:.3f}')
                if avg_return >= success_ret:
                    print('Решено!')
                    break

            s, _ = env.reset()
            done, episode_return = False, 0.
            eval = False

run_actor_critic(eval_schedule=2000, total_max_steps=100_000)

global_step=2006 | avg_return=9.000
global_step=4000 | avg_return=9.000
global_step=6005 | avg_return=9.000
global_step=8007 | avg_return=16.250
global_step=10098 | avg_return=34.000
global_step=12102 | avg_return=45.667
global_step=14000 | avg_return=52.000
global_step=16018 | avg_return=61.625
global_step=18011 | avg_return=71.000
global_step=20032 | avg_return=75.700
global_step=22058 | avg_return=85.900
global_step=24052 | avg_return=95.400
global_step=26086 | avg_return=107.700
global_step=28080 | avg_return=123.800
global_step=30019 | avg_return=128.700
global_step=32106 | avg_return=135.200
global_step=34017 | avg_return=142.100
global_step=36043 | avg_return=147.500
global_step=38083 | avg_return=159.200
global_step=40004 | avg_return=163.400
global_step=42039 | avg_return=163.700
global_step=44137 | avg_return=175.300
global_step=46022 | avg_return=182.900
global_step=48028 | avg_return=174.800
global_step=50068 | avg_return=173.300
global_step=52114 | avg_return=172.700
globa